In [2]:
import os
import argparse
import sys
import mne
import math
import time
import json
import numpy as np
from scipy.signal import butter, filtfilt
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef


start = time.time()

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

if is_notebook():
    args = argparse.Namespace(s=None, c=None)
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', default=None)
    parser.add_argument('-c', default=None, type=int)
    args = parser.parse_args()

print(args.s)
print(args.c)
print(__doc__)



##------------------------------------------------------------------------------
# Filtering the EEG Data:
def apply_filter(data, b, a):
    r = filtfilt(b=b, a=a, x=data)
    return r
##------------------------------------------------------------------------------
#Setting Up Paths and Directories
repository_base = os.path.dirname(os.path.dirname(os.path.abspath('C:/Users/imran/dataverse_files/results/sub-K_classification_scores.json')))

base = os.path.join(repository_base, "eeg")
save_base = os.path.join(repository_base, "results")

if not os.path.exists(save_base):
    os.makedirs(save_base)

# Subject and test class selection
subject = 'sub-K'
if args.s is not None:
    subject = args.s
test_class =3
if args.c is not None:
    test_class = args.c

# Sampling frequency and filter parameters
Fs = 1000
fc = [1, 40]
b, a = butter(N=2, Wn=np.array(fc) / (Fs / 2), btype='bandpass', output='ba')

# Function to apply the filter
def apply_filter(data, b, a):
    return filtfilt(b=b, a=a, x=data)

# Load data based on class
target_file = []
non_target_file = []

fnum = np.array([[1, 4],
                 [2, 5],
                 [3, 6]])

trig_id = [2, 8, 32]
tasks = ['low', 'low', 'mid', 'mid', 'high', 'high']
reject = {'eeg': 100e-6, 'eog': 500e-6}

#==================================================================================================================

t = []
nt = []

target = []
non_target = []

# Iterate through tasks to load the corresponding EEG files
for i in range(len(fnum.ravel())):
    fname = os.path.join(base, subject, "eeg", "%s_task-%s_run-%d_eeg.vhdr" % (subject, tasks[i], fnum.ravel()[i]))
    if np.any(fnum[test_class - 1] == fnum.ravel()[i]):
        if isinstance(target_file, list):
            target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            target_file = target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            target_file = mne.concatenate_raws([target_file, tmp])
    else:
        if isinstance(non_target_file, list):
            non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            non_target_file = non_target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            non_target_file = mne.concatenate_raws([non_target_file, tmp])

# Create events for target and non-target stimuli
event_id = {'target_stimulus_id': -100, 'non_target_stimulus_id': -5100}
target_eve = mne.events_from_annotations(target_file)
target_eve = mne.merge_events(target_eve[0], [trig_id[test_class - 1]], event_id['target_stimulus_id'], replace_events=True)
non_target_eve = mne.events_from_annotations(non_target_file)
non_target_eve = mne.merge_events(non_target_eve[0], [trig_id[test_class - 1]], event_id['non_target_stimulus_id'], replace_events=True)

# Epoching for target and non-target events
tmin, tmax = -0.02, 1.4
baseline = (0.0, 0.01)
target_epochs = mne.Epochs(target_file, events=target_eve, event_id=event_id['target_stimulus_id'], tmin=tmin, tmax=tmax, baseline=baseline, reject=reject, preload=True)

#=======================================================================Sliding Window Epoching =================================================================================================================
import numpy as np

# Assuming data = target_epochs.get_data()
# data shape: (n_epochs, n_channels, n_times)

def find_max_peak_window(data, Fs, window_size_ms=600, overlap_size_ms=200, epoch_index=0):
    """
    Function to find the window with the maximum peak value within a specific epoch and return its start and end indices.
    """
    window_size_samples = int(window_size_ms * Fs / 1000)  # Convert window size from ms to samples
    overlap_size_samples = int(overlap_size_ms * Fs / 1000)  # Convert overlap size from ms to samples

    # Select the epoch to analyze
    epoch_data = data[epoch_index]  # Shape: (n_channels, n_times)
    n_channels, n_times = epoch_data.shape  # Get the number of channels and time points

    # Initialize variables to store the maximum peak value and corresponding window indices
    max_peak_value = -np.inf
    max_peak_start_index = None
    max_peak_end_index = None

    # Initialize start of window
    start = 0

    # Loop through the epoch using sliding windows with overlap
    while start + window_size_samples <= n_times:
        # Extract the current window of data
        window_data = epoch_data[:, start:start + window_size_samples]

        # Calculate the overall peak value in this window (across all channels)
        current_peak_value = np.max(window_data)  # Overall peak value across all channels

        # Check if this is the highest peak we've found in this epoch
        if current_peak_value > max_peak_value:
            max_peak_value = current_peak_value
            max_peak_start_index = start
            max_peak_end_index = start + window_size_samples - 1  # End index of the window

        # Move to the next window
        start += (window_size_samples - overlap_size_samples)

    # Print the maximum peak value and its window indices for the epoch
    #print(f"Epoch {epoch_index + 1} - Max Peak Value: {max_peak_value}")
    #print(f"Epoch {epoch_index + 1} - Window Start Index: {max_peak_start_index}, End Index: {max_peak_end_index}")

    # Return the start and end index for re-epoching
    return max_peak_start_index, max_peak_end_index

def analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200):
    """
    Function to find and print the maximum peak values and corresponding window indices across all epochs.
    """
    n_epochs = data.shape[0]  # Get the number of epochs

    # Initialize a list to store start and end indices for each epoch
    peak_window_indices = []

    # Loop through all epochs
    for epoch_index in range(n_epochs):
        print(f"\nAnalyzing Epoch {epoch_index + 1}")
        start_index, end_index = find_max_peak_window(data, Fs, window_size_ms, overlap_size_ms, epoch_index)
        peak_window_indices.append((start_index, end_index))
    return peak_window_indices

def re_epoch_data(data, peak_window_indices):
    """
    Function to re-epoch the data based on the peak window indices.
    """
    re_epoched_data = []

    # Loop through each epoch and apply re-epoching using peak window indices
    for epoch_index, (start, end) in enumerate(peak_window_indices):
        re_epoched_epoch_data = data[epoch_index][:, start:end + 1]  # Extract data within peak window
        re_epoched_data.append(re_epoched_epoch_data)

    # Convert re-epoched data to numpy array for easier manipulation
    re_epoched_data = np.array(re_epoched_data)

    #print("\nRe-epoching complete.")
    #print("Re-epoched data shape:", re_epoched_data.shape)
    return re_epoched_data
# Example usage
Fs = 1000  # Example sampling frequency in Hz
data = target_epochs.get_data()  # Load your EEG data from the epochs

# Step 1: Find peak window indices for each epoch
peak_window_indices = analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200)

# Step 2: Re-epoch the data based on the peak window indices
re_epoched_data = re_epoch_data(data, peak_window_indices)


from mne import EpochsArray
from mne import create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs:", re_epoched_epochs)



from mne import EpochsArray, create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

#Apply baseline correction (-0.00 to 0.000 seconds) and filter (highpass=0.50 Hz, lowpass=100.00 Hz)
baseline=(0.0,0.01)
re_epoched_epochs.apply_baseline(baseline)

# Apply bandpass filtering (highpass = 0.50 Hz, lowpass = 100.00 Hz)
re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.")


import mne
# Defining time windows for analysis
tmin, tmax = -0.0, 0.599
baseline=(0.0,0.01)

# Creating epochs
non_target_epochs = mne.Epochs(
    non_target_file,
    events=non_target_eve,
    event_id=event_id['non_target_stimulus_id'],
    tmin=tmin,
    tmax=tmax,
    baseline=baseline,
    reject=reject,
    preload=True
)

# Define new channel names
new_channel_names = [f'chan{i}' for i in range(len(non_target_epochs.ch_names))]

# Create a dictionary to map old channel names to new channel names
rename_dict = dict(zip(non_target_epochs.ch_names, new_channel_names))

# Rename channels
non_target_epochs.rename_channels(rename_dict)

# Now `non_target_epochs` has the new channel names
print("Updated channel names:", non_target_epochs.ch_names)


target_epochs=re_epoched_epochs


# Drop last two channels from target_epochs and non_target_epochs
target_epochs.drop_channels(target_epochs.info['ch_names'][-2:])
non_target_epochs.drop_channels(non_target_epochs.info['ch_names'][-2:])

# Concatenate target_epochs and non_target_epochs
epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])

# Prepare data and labels for classification
epochs_data = epochs.get_data()  # Extract EEG data as a numpy array
labels = epochs.events[:, -1]    # Extract event IDs as labels


None
None
Automatically created module for IPython interactive environment
Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-1_eeg.vhdr...
Setting channel info structure...


C:\Users\imran\AppData\Local\Temp\ipykernel_15828\3622151304.py:111: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_15828\3622151304.py:111: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Reading 0 ... 320479  =      0.000 ...   320.479 secs...
Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-4_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 314999  =      0.000 ...   314.999 secs...


C:\Users\imran\AppData\Local\Temp\ipykernel_15828\3622151304.py:114: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_15828\3622151304.py:114: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-mid_run-2_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 324079  =      0.000 ...   324.079 secs...


...

C:\Users\imran\AppData\Local\Temp\ipykernel_15828\3622151304.py:248: RuntimeWarning: filter_length (6601) is longer than the signal (600), distortion is likely. Reduce filter length or filter a longer signal.
  re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    1.1s

Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.
Not setting metadata


[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    1.7s


238 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 238 events and 600 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5', 'F3', 'F2', 'F4', 'F6']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8

C:\Users\imran\AppData\Local\Temp\ipykernel_15828\3622151304.py:292: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])


In [5]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        
        clf = SVC(kernel='rbf', C=1.0)
        clf.fit(X_train_csp, y_train)
        predictions = clf.predict(X_val_csp)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)  # Binary Grey Wolf Optimization with 10-Fold Cross-Validation

def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(iteration_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
        
        # Print selected feature count and fitness per iteration
        selected_features_per_iteration = [np.sum(wolf) for wolf in wolves]
        print(f"Iteration {iteration + 1}: Selected Features Count: {selected_features_per_iteration}, Fitness Values: {iteration_fitness}")
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=3, max_iter=2
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")


Computing rank from data with rank=None
    Using tolerance 7e-05 (2.2e-16 eps * 22 dim * 1.4e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.2e-05 (2.2e-16 eps * 22 dim * 1.5e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.1e-05 (2.2e-16 eps * 22 dim * 1.5e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance

In [4]:
selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")


Selected Lobes Information:
Frontal lobe: Selected Channels: [0, 5, 6, 7, 8, 9, 12, 13, 16, 17, 18, 19]
Temporal lobe: Selected Channels: [22, 23, 25, 27, 28, 30, 32, 34, 37]
Parietal lobe: Selected Channels: [38, 39, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 53]
Occipital lobe: Selected Channels: [56, 59, 61, 62, 63]


# Stream - 1

In [1]:
import os
import argparse
import sys
import mne
import math
import time
import json
import numpy as np
from scipy.signal import butter, filtfilt
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef


start = time.time()

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

if is_notebook():
    args = argparse.Namespace(s=None, c=None)
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', default=None)
    parser.add_argument('-c', default=None, type=int)
    args = parser.parse_args()

print(args.s)
print(args.c)
print(__doc__)



##------------------------------------------------------------------------------
# Filtering the EEG Data:
def apply_filter(data, b, a):
    r = filtfilt(b=b, a=a, x=data)
    return r
##------------------------------------------------------------------------------
#Setting Up Paths and Directories
repository_base = os.path.dirname(os.path.dirname(os.path.abspath('C:/Users/imran/dataverse_files/results/sub-K_classification_scores.json')))

base = os.path.join(repository_base, "eeg")
save_base = os.path.join(repository_base, "results")

if not os.path.exists(save_base):
    os.makedirs(save_base)

# Subject and test class selection
subject = 'sub-K'
if args.s is not None:
    subject = args.s
test_class =1
if args.c is not None:
    test_class = args.c

# Sampling frequency and filter parameters
Fs = 1000
fc = [1, 40]
b, a = butter(N=2, Wn=np.array(fc) / (Fs / 2), btype='bandpass', output='ba')

# Function to apply the filter
def apply_filter(data, b, a):
    return filtfilt(b=b, a=a, x=data)

# Load data based on class
target_file = []
non_target_file = []

fnum = np.array([[1, 4],
                 [2, 5],
                 [3, 6]])

trig_id = [2, 8, 32]
tasks = ['low', 'low', 'mid', 'mid', 'high', 'high']
reject = {'eeg': 100e-6, 'eog': 500e-6}


#==================================================================================================================

t = []
nt = []

target = []
non_target = []

# Iterate through tasks to load the corresponding EEG files
for i in range(len(fnum.ravel())):
    fname = os.path.join(base, subject, "eeg", "%s_task-%s_run-%d_eeg.vhdr" % (subject, tasks[i], fnum.ravel()[i]))
    if np.any(fnum[test_class - 1] == fnum.ravel()[i]):
        if isinstance(target_file, list):
            target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            target_file = target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            target_file = mne.concatenate_raws([target_file, tmp])
    else:
        if isinstance(non_target_file, list):
            non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            non_target_file = non_target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            non_target_file = mne.concatenate_raws([non_target_file, tmp])

# Create events for target and non-target stimuli
event_id = {'target_stimulus_id': -100, 'non_target_stimulus_id': -5100}
target_eve = mne.events_from_annotations(target_file)
target_eve = mne.merge_events(target_eve[0], [trig_id[test_class - 1]], event_id['target_stimulus_id'], replace_events=True)
non_target_eve = mne.events_from_annotations(non_target_file)
non_target_eve = mne.merge_events(non_target_eve[0], [trig_id[test_class - 1]], event_id['non_target_stimulus_id'], replace_events=True)

# Epoching for target and non-target events
tmin, tmax = -0.02, 1.4
baseline = (0.0, 0.1)
target_epochs = mne.Epochs(target_file, events=target_eve, event_id=event_id['target_stimulus_id'], tmin=tmin, tmax=tmax, baseline=baseline, reject=reject, preload=True)

#=======================================================================Sliding Window Epoching =================================================================================================================

import numpy as np

# Assuming data = target_epochs.get_data()
# data shape: (n_epochs, n_channels, n_times)

def find_max_peak_window(data, Fs, window_size_ms=600, overlap_size_ms=200, epoch_index=0):
    """
    Function to find the window with the maximum peak value within a specific epoch and return its start and end indices.
    """
    window_size_samples = int(window_size_ms * Fs / 1000)  # Convert window size from ms to samples
    overlap_size_samples = int(overlap_size_ms * Fs / 1000)  # Convert overlap size from ms to samples

    # Select the epoch to analyze
    epoch_data = data[epoch_index]  # Shape: (n_channels, n_times)
    n_channels, n_times = epoch_data.shape  # Get the number of channels and time points

    # Initialize variables to store the maximum peak value and corresponding window indices
    max_peak_value = -np.inf
    max_peak_start_index = None
    max_peak_end_index = None

    # Initialize start of window
    start = 0

    # Loop through the epoch using sliding windows with overlap
    while start + window_size_samples <= n_times:
        # Extract the current window of data
        window_data = epoch_data[:, start:start + window_size_samples]

        # Calculate the overall peak value in this window (across all channels)
        current_peak_value = np.max(window_data)  # Overall peak value across all channels

        # Check if this is the highest peak we've found in this epoch
        if current_peak_value > max_peak_value:
            max_peak_value = current_peak_value
            max_peak_start_index = start
            max_peak_end_index = start + window_size_samples - 1  # End index of the window

        # Move to the next window
        start += (window_size_samples - overlap_size_samples)

    # Print the maximum peak value and its window indices for the epoch
    #print(f"Epoch {epoch_index + 1} - Max Peak Value: {max_peak_value}")
    #print(f"Epoch {epoch_index + 1} - Window Start Index: {max_peak_start_index}, End Index: {max_peak_end_index}")

    # Return the start and end index for re-epoching
    return max_peak_start_index, max_peak_end_index

def analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200):
    """
    Function to find and print the maximum peak values and corresponding window indices across all epochs.
    """
    n_epochs = data.shape[0]  # Get the number of epochs

    # Initialize a list to store start and end indices for each epoch
    peak_window_indices = []

    # Loop through all epochs
    for epoch_index in range(n_epochs):
        print(f"\nAnalyzing Epoch {epoch_index + 1}")
        start_index, end_index = find_max_peak_window(data, Fs, window_size_ms, overlap_size_ms, epoch_index)
        peak_window_indices.append((start_index, end_index))
    return peak_window_indices

def re_epoch_data(data, peak_window_indices):
    """
    Function to re-epoch the data based on the peak window indices.
    """
    re_epoched_data = []

    # Loop through each epoch and apply re-epoching using peak window indices
    for epoch_index, (start, end) in enumerate(peak_window_indices):
        re_epoched_epoch_data = data[epoch_index][:, start:end + 1]  # Extract data within peak window
        re_epoched_data.append(re_epoched_epoch_data)

    # Convert re-epoched data to numpy array for easier manipulation
    re_epoched_data = np.array(re_epoched_data)

    #print("\nRe-epoching complete.")
    #print("Re-epoched data shape:", re_epoched_data.shape)
    return re_epoched_data

# Example usage
Fs = 1000  # Example sampling frequency in Hz
data = target_epochs.get_data()  # Load your EEG data from the epochs

# Step 1: Find peak window indices for each epoch
peak_window_indices = analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200)

# Step 2: Re-epoch the data based on the peak window indices
re_epoched_data = re_epoch_data(data, peak_window_indices)


from mne import EpochsArray
from mne import create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs:", re_epoched_epochs)



from mne import EpochsArray, create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

#Apply baseline correction (-0.00 to 0.000 seconds) and filter (highpass=0.50 Hz, lowpass=100.00 Hz)
baseline=(0.0,0.1)
re_epoched_epochs.apply_baseline(baseline)

# Apply bandpass filtering (highpass = 0.50 Hz, lowpass = 100.00 Hz)
re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.")




import mne
# Defining time windows for analysis
tmin, tmax = -0.0, 0.599
baseline = (0.0, 0.1)

# Creating epochs
non_target_epochs = mne.Epochs(
    non_target_file,
    events=non_target_eve,
    event_id=event_id['non_target_stimulus_id'],
    tmin=tmin,
    tmax=tmax,
    baseline=baseline,
    reject=reject,
    preload=True
)

# Define new channel names
new_channel_names = [f'chan{i}' for i in range(len(non_target_epochs.ch_names))]

# Create a dictionary to map old channel names to new channel names
rename_dict = dict(zip(non_target_epochs.ch_names, new_channel_names))

# Rename channels
non_target_epochs.rename_channels(rename_dict)

# Now `non_target_epochs` has the new channel names
print("Updated channel names:", non_target_epochs.ch_names)


target_epochs=re_epoched_epochs


# Drop last two channels from target_epochs and non_target_epochs
target_epochs.drop_channels(target_epochs.info['ch_names'][-2:])
non_target_epochs.drop_channels(non_target_epochs.info['ch_names'][-2:])

# Concatenate target_epochs and non_target_epochs
epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])

# Prepare data and labels for classification
epochs_data = epochs.get_data()  # Extract EEG data as a numpy array
labels = epochs.events[:, -1]    # Extract event IDs as labels


#======================================================== SVM =============================================

import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        
        clf = SVC(kernel='rbf', C=1.0)
        clf.fit(X_train_csp, y_train)
        predictions = clf.predict(X_val_csp)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)

# Binary Grey Wolf Optimization with 10-Fold Cross-Validation
def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            # Handling Hierarchy (Alpha, Beta, Delta)
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(alpha_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                # Encircling and Position Update Equations
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                #Binary Update with Sigmoid Function
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=4, max_iter=20
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")

None
None
Automatically created module for IPython interactive environment
Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-1_eeg.vhdr...
Setting channel info structure...


C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1295596528.py:104: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1295596528.py:104: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Reading 0 ... 320479  =      0.000 ...   320.479 secs...
Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-4_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 314999  =      0.000 ...   314.999 secs...


C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1295596528.py:107: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1295596528.py:107: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-mid_run-2_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 324079  =      0.000 ...   324.079 secs...


...

Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.
Not setting metadata


[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.8s


242 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 242 events and 600 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF3', 'AFz']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F6']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F2', 'F4']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7'

C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1295596528.py:297: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])


    Using tolerance 5.9e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.9e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=

# Stream 2

In [2]:
import os
import argparse
import sys
import mne
import math
import time
import json
import numpy as np
from scipy.signal import butter, filtfilt
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef


start = time.time()

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

if is_notebook():
    args = argparse.Namespace(s=None, c=None)
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', default=None)
    parser.add_argument('-c', default=None, type=int)
    args = parser.parse_args()

print(args.s)
print(args.c)
print(__doc__)



##------------------------------------------------------------------------------
# Filtering the EEG Data:
def apply_filter(data, b, a):
    r = filtfilt(b=b, a=a, x=data)
    return r
##------------------------------------------------------------------------------
#Setting Up Paths and Directories
repository_base = os.path.dirname(os.path.dirname(os.path.abspath('C:/Users/imran/dataverse_files/results/sub-K_classification_scores.json')))

base = os.path.join(repository_base, "eeg")
save_base = os.path.join(repository_base, "results")

if not os.path.exists(save_base):
    os.makedirs(save_base)

# Subject and test class selection
subject = 'sub-K'
if args.s is not None:
    subject = args.s
test_class =2
if args.c is not None:
    test_class = args.c

# Sampling frequency and filter parameters
Fs = 1000
fc = [1, 40]
b, a = butter(N=2, Wn=np.array(fc) / (Fs / 2), btype='bandpass', output='ba')

# Function to apply the filter
def apply_filter(data, b, a):
    return filtfilt(b=b, a=a, x=data)

# Load data based on class
target_file = []
non_target_file = []

fnum = np.array([[1, 4],
                 [2, 5],
                 [3, 6]])

trig_id = [2, 8, 32]
tasks = ['low', 'low', 'mid', 'mid', 'high', 'high']
reject = {'eeg': 100e-6, 'eog': 500e-6}


#==================================================================================================================

t = []
nt = []

target = []
non_target = []

# Iterate through tasks to load the corresponding EEG files
for i in range(len(fnum.ravel())):
    fname = os.path.join(base, subject, "eeg", "%s_task-%s_run-%d_eeg.vhdr" % (subject, tasks[i], fnum.ravel()[i]))
    if np.any(fnum[test_class - 1] == fnum.ravel()[i]):
        if isinstance(target_file, list):
            target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            target_file = target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            target_file = mne.concatenate_raws([target_file, tmp])
    else:
        if isinstance(non_target_file, list):
            non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            non_target_file = non_target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            non_target_file = mne.concatenate_raws([non_target_file, tmp])

# Create events for target and non-target stimuli
event_id = {'target_stimulus_id': -100, 'non_target_stimulus_id': -5100}
target_eve = mne.events_from_annotations(target_file)
target_eve = mne.merge_events(target_eve[0], [trig_id[test_class - 1]], event_id['target_stimulus_id'], replace_events=True)
non_target_eve = mne.events_from_annotations(non_target_file)
non_target_eve = mne.merge_events(non_target_eve[0], [trig_id[test_class - 1]], event_id['non_target_stimulus_id'], replace_events=True)

# Epoching for target and non-target events
tmin, tmax = -0.02, 1.4
baseline = (0.0, 0.1)
target_epochs = mne.Epochs(target_file, events=target_eve, event_id=event_id['target_stimulus_id'], tmin=tmin, tmax=tmax, baseline=baseline, reject=reject, preload=True)

#=======================================================================Sliding Window Epoching =================================================================================================================

import numpy as np

# Assuming data = target_epochs.get_data()
# data shape: (n_epochs, n_channels, n_times)

def find_max_peak_window(data, Fs, window_size_ms=600, overlap_size_ms=200, epoch_index=0):
    """
    Function to find the window with the maximum peak value within a specific epoch and return its start and end indices.
    """
    window_size_samples = int(window_size_ms * Fs / 1000)  # Convert window size from ms to samples
    overlap_size_samples = int(overlap_size_ms * Fs / 1000)  # Convert overlap size from ms to samples

    # Select the epoch to analyze
    epoch_data = data[epoch_index]  # Shape: (n_channels, n_times)
    n_channels, n_times = epoch_data.shape  # Get the number of channels and time points

    # Initialize variables to store the maximum peak value and corresponding window indices
    max_peak_value = -np.inf
    max_peak_start_index = None
    max_peak_end_index = None

    # Initialize start of window
    start = 0

    # Loop through the epoch using sliding windows with overlap
    while start + window_size_samples <= n_times:
        # Extract the current window of data
        window_data = epoch_data[:, start:start + window_size_samples]

        # Calculate the overall peak value in this window (across all channels)
        current_peak_value = np.max(window_data)  # Overall peak value across all channels

        # Check if this is the highest peak we've found in this epoch
        if current_peak_value > max_peak_value:
            max_peak_value = current_peak_value
            max_peak_start_index = start
            max_peak_end_index = start + window_size_samples - 1  # End index of the window

        # Move to the next window
        start += (window_size_samples - overlap_size_samples)

    # Print the maximum peak value and its window indices for the epoch
    #print(f"Epoch {epoch_index + 1} - Max Peak Value: {max_peak_value}")
    #print(f"Epoch {epoch_index + 1} - Window Start Index: {max_peak_start_index}, End Index: {max_peak_end_index}")

    # Return the start and end index for re-epoching
    return max_peak_start_index, max_peak_end_index

def analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200):
    """
    Function to find and print the maximum peak values and corresponding window indices across all epochs.
    """
    n_epochs = data.shape[0]  # Get the number of epochs

    # Initialize a list to store start and end indices for each epoch
    peak_window_indices = []

    # Loop through all epochs
    for epoch_index in range(n_epochs):
        print(f"\nAnalyzing Epoch {epoch_index + 1}")
        start_index, end_index = find_max_peak_window(data, Fs, window_size_ms, overlap_size_ms, epoch_index)
        peak_window_indices.append((start_index, end_index))
    return peak_window_indices

def re_epoch_data(data, peak_window_indices):
    """
    Function to re-epoch the data based on the peak window indices.
    """
    re_epoched_data = []

    # Loop through each epoch and apply re-epoching using peak window indices
    for epoch_index, (start, end) in enumerate(peak_window_indices):
        re_epoched_epoch_data = data[epoch_index][:, start:end + 1]  # Extract data within peak window
        re_epoched_data.append(re_epoched_epoch_data)

    # Convert re-epoched data to numpy array for easier manipulation
    re_epoched_data = np.array(re_epoched_data)

    #print("\nRe-epoching complete.")
    #print("Re-epoched data shape:", re_epoched_data.shape)
    return re_epoched_data

# Example usage
Fs = 1000  # Example sampling frequency in Hz
data = target_epochs.get_data()  # Load your EEG data from the epochs

# Step 1: Find peak window indices for each epoch
peak_window_indices = analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200)

# Step 2: Re-epoch the data based on the peak window indices
re_epoched_data = re_epoch_data(data, peak_window_indices)


from mne import EpochsArray
from mne import create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs:", re_epoched_epochs)



from mne import EpochsArray, create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

#Apply baseline correction (-0.00 to 0.000 seconds) and filter (highpass=0.50 Hz, lowpass=100.00 Hz)
baseline=(0.0,0.1)
re_epoched_epochs.apply_baseline(baseline)

# Apply bandpass filtering (highpass = 0.50 Hz, lowpass = 100.00 Hz)
re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.")




import mne
# Defining time windows for analysis
tmin, tmax = -0.0, 0.599
#baseline=(0.0,0.01)

# Creating epochs
non_target_epochs = mne.Epochs(
    non_target_file,
    events=non_target_eve,
    event_id=event_id['non_target_stimulus_id'],
    tmin=tmin,
    tmax=tmax,
    baseline=baseline,
    reject=reject,
    preload=True
)

# Define new channel names
new_channel_names = [f'chan{i}' for i in range(len(non_target_epochs.ch_names))]

# Create a dictionary to map old channel names to new channel names
rename_dict = dict(zip(non_target_epochs.ch_names, new_channel_names))

# Rename channels
non_target_epochs.rename_channels(rename_dict)

# Now `non_target_epochs` has the new channel names
print("Updated channel names:", non_target_epochs.ch_names)


target_epochs=re_epoched_epochs


# Drop last two channels from target_epochs and non_target_epochs
target_epochs.drop_channels(target_epochs.info['ch_names'][-2:])
non_target_epochs.drop_channels(non_target_epochs.info['ch_names'][-2:])

# Concatenate target_epochs and non_target_epochs
epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])

# Prepare data and labels for classification
epochs_data = epochs.get_data()  # Extract EEG data as a numpy array
labels = epochs.events[:, -1]    # Extract event IDs as labels


#======================================================== SVM =============================================

import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        
        clf = SVC(kernel='rbf', C=1.0)
        clf.fit(X_train_csp, y_train)
        predictions = clf.predict(X_val_csp)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)

# Binary Grey Wolf Optimization with 10-Fold Cross-Validation
def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            # Handling Hierarchy (Alpha, Beta, Delta)
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(alpha_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                # Encircling and Position Update Equations
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                #Binary Update with Sigmoid Function
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=3, max_iter=20
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")

None
None
Automatically created module for IPython interactive environment
Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-1_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 320479  =      0.000 ...   320.479 secs...


C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1014258282.py:112: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1014258282.py:112: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-4_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 314999  =      0.000 ...   314.999 secs...


C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1014258282.py:115: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1014258282.py:115: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-mid_run-2_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 324079  =      0.000 ...   324.079 secs...


...

Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.
Not setting metadata
236 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 236 events and 600 original time points ...
    Rejecting  epoch based on EEG : ['FT8']


[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    1.0s


    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5', 'F3']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['AF8']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejectin

C:\Users\imran\AppData\Local\Temp\ipykernel_2148\1014258282.py:297: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])


    Using tolerance 5.3e-05 (2.2e-16 eps * 22 dim * 1.1e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e-05 (2.2e-16 eps * 22 dim * 1.1e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.3e-05 (2.2e-16 eps * 22 dim * 1.1e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating clas

# Stream 3

In [33]:
import os
import argparse
import sys
import mne
import math
import time
import json
import numpy as np
from scipy.signal import butter, filtfilt
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef


start = time.time()

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

if is_notebook():
    args = argparse.Namespace(s=None, c=None)
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', default=None)
    parser.add_argument('-c', default=None, type=int)
    args = parser.parse_args()

print(args.s)
print(args.c)
print(__doc__)

## ------------------------------------------------------------------------------
# Filtering the EEG Data:
def apply_filter(data, b, a):
    r = filtfilt(b=b, a=a, x=data)
    return r
##------------------------------------------------------------------------------
#Setting Up Paths and Directories
repository_base = os.path.dirname(os.path.dirname(os.path.abspath('C:/Users/imran/dataverse_files/results/sub-K_classification_scores.json')))

base = os.path.join(repository_base, "eeg")
save_base = os.path.join(repository_base, "results")

if not os.path.exists(save_base):
    os.makedirs(save_base)

# Subject and test class selection
subject = 'sub-K'
if args.s is not None:
    subject = args.s
test_class =3
if args.c is not None:
    test_class = args.c

# Sampling frequency and filter parameters
Fs = 1000
fc = [1, 40]
b, a = butter(N=2, Wn=np.array(fc) / (Fs / 2), btype='bandpass', output='ba')

# Function to apply the filter
def apply_filter(data, b, a):
    return filtfilt(b=b, a=a, x=data)

# Load data based on class
target_file = []
non_target_file = []

fnum = np.array([[1, 4],
                 [2, 5],
                 [3, 6]])

trig_id = [2, 8, 32]
tasks = ['low', 'low', 'mid', 'mid', 'high', 'high']
reject = {'eeg': 100e-6, 'eog': 500e-6}


#==================================================================================================================

t = []
nt = []

target = []
non_target = []

# Iterate through tasks to load the corresponding EEG files
for i in range(len(fnum.ravel())):
    fname = os.path.join(base, subject, "eeg", "%s_task-%s_run-%d_eeg.vhdr" % (subject, tasks[i], fnum.ravel()[i]))
    if np.any(fnum[test_class - 1] == fnum.ravel()[i]):
        if isinstance(target_file, list):
            target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            target_file = target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            target_file = mne.concatenate_raws([target_file, tmp])
    else:
        if isinstance(non_target_file, list):
            non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            non_target_file = non_target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            non_target_file = mne.concatenate_raws([non_target_file, tmp])

# Create events for target and non-target stimuli
event_id = {'target_stimulus_id': -100, 'non_target_stimulus_id': -5100}
target_eve = mne.events_from_annotations(target_file)
target_eve = mne.merge_events(target_eve[0], [trig_id[test_class - 1]], event_id['target_stimulus_id'], replace_events=True)
non_target_eve = mne.events_from_annotations(non_target_file)
non_target_eve = mne.merge_events(non_target_eve[0], [trig_id[test_class - 1]], event_id['non_target_stimulus_id'], replace_events=True)

# Epoching for target and non-target events
tmin, tmax = -0.02, 1.4
baseline = (0.0, 0.1)
target_epochs = mne.Epochs(target_file, events=target_eve, event_id=event_id['target_stimulus_id'], tmin=tmin, tmax=tmax, baseline=baseline, reject=reject, preload=True)

#=======================================================================Sliding Window Epoching =================================================================================================================

import numpy as np

# Assuming data = target_epochs.get_data()
# data shape: (n_epochs, n_channels, n_times)

def find_max_peak_window(data, Fs, window_size_ms=600, overlap_size_ms=200, epoch_index=0):
    """
    Function to find the window with the maximum peak value within a specific epoch and return its start and end indices.
    """
    window_size_samples = int(window_size_ms * Fs / 1000)  # Convert window size from ms to samples
    overlap_size_samples = int(overlap_size_ms * Fs / 1000)  # Convert overlap size from ms to samples

    # Select the epoch to analyze
    epoch_data = data[epoch_index]  # Shape: (n_channels, n_times)
    n_channels, n_times = epoch_data.shape  # Get the number of channels and time points

    # Initialize variables to store the maximum peak value and corresponding window indices
    max_peak_value = -np.inf
    max_peak_start_index = None
    max_peak_end_index = None

    # Initialize start of window
    start = 0

    # Loop through the epoch using sliding windows with overlap
    while start + window_size_samples <= n_times:
        # Extract the current window of data
        window_data = epoch_data[:, start:start + window_size_samples]

        # Calculate the overall peak value in this window (across all channels)
        current_peak_value = np.max(window_data)  # Overall peak value across all channels

        # Check if this is the highest peak we've found in this epoch
        if current_peak_value > max_peak_value:
            max_peak_value = current_peak_value
            max_peak_start_index = start
            max_peak_end_index = start + window_size_samples - 1  # End index of the window

        # Move to the next window
        start += (window_size_samples - overlap_size_samples)

    # Print the maximum peak value and its window indices for the epoch
    #print(f"Epoch {epoch_index + 1} - Max Peak Value: {max_peak_value}")
    #print(f"Epoch {epoch_index + 1} - Window Start Index: {max_peak_start_index}, End Index: {max_peak_end_index}")

    # Return the start and end index for re-epoching
    return max_peak_start_index, max_peak_end_index

def analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200):
    """
    Function to find and print the maximum peak values and corresponding window indices across all epochs.
    """
    n_epochs = data.shape[0]  # Get the number of epochs

    # Initialize a list to store start and end indices for each epoch
    peak_window_indices = []

    # Loop through all epochs
    for epoch_index in range(n_epochs):
        print(f"\nAnalyzing Epoch {epoch_index + 1}")
        start_index, end_index = find_max_peak_window(data, Fs, window_size_ms, overlap_size_ms, epoch_index)
        peak_window_indices.append((start_index, end_index))
    return peak_window_indices

def re_epoch_data(data, peak_window_indices):
    """
    Function to re-epoch the data based on the peak window indices.
    """
    re_epoched_data = []

    # Loop through each epoch and apply re-epoching using peak window indices
    for epoch_index, (start, end) in enumerate(peak_window_indices):
        re_epoched_epoch_data = data[epoch_index][:, start:end + 1]  # Extract data within peak window
        re_epoched_data.append(re_epoched_epoch_data)

    # Convert re-epoched data to numpy array for easier manipulation
    re_epoched_data = np.array(re_epoched_data)

    #print("\nRe-epoching complete.")
    #print("Re-epoched data shape:", re_epoched_data.shape)
    return re_epoched_data

# Example usage
Fs = 1000  # Example sampling frequency in Hz
data = target_epochs.get_data()  # Load your EEG data from the epochs

# Step 1: Find peak window indices for each epoch
peak_window_indices = analyze_all_epochs_max_peaks(data, Fs, window_size_ms=600, overlap_size_ms=200)

# Step 2: Re-epoch the data based on the peak window indices
re_epoched_data = re_epoch_data(data, peak_window_indices)


from mne import EpochsArray
from mne import create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs:", re_epoched_epochs)



from mne import EpochsArray, create_info

# Create MNE-compatible epochs with `re_epoched_data`
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)

#Apply baseline correction (-0.00 to 0.000 seconds) and filter (highpass=0.50 Hz, lowpass=100.00 Hz)
baseline=(0.0,0.1)
re_epoched_epochs.apply_baseline(baseline)

# Apply bandpass filtering (highpass = 0.50 Hz, lowpass = 100.00 Hz)
re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)

# Now `re_epoched_epochs` is ready for further analysis
print("Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.")




import mne
# Defining time windows for analysis
tmin, tmax = -0.0, 0.599
baseline=(0.0,0.1)

# Creating epochs
non_target_epochs = mne.Epochs(
    non_target_file,
    events=non_target_eve,
    event_id=event_id['non_target_stimulus_id'],
    tmin=tmin,
    tmax=tmax,
    baseline=baseline,
    reject=reject,
    preload=True
)

# Define new channel names
new_channel_names = [f'chan{i}' for i in range(len(non_target_epochs.ch_names))]

# Create a dictionary to map old channel names to new channel names
rename_dict = dict(zip(non_target_epochs.ch_names, new_channel_names))

# Rename channels
non_target_epochs.rename_channels(rename_dict)

# Now `non_target_epochs` has the new channel names
print("Updated channel names:", non_target_epochs.ch_names)


target_epochs=re_epoched_epochs


# Drop last two channels from target_epochs and non_target_epochs
target_epochs.drop_channels(target_epochs.info['ch_names'][-2:])
non_target_epochs.drop_channels(non_target_epochs.info['ch_names'][-2:])

# Concatenate target_epochs and non_target_epochs
epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])

# Prepare data and labels for classification
epochs_data = epochs.get_data()  # Extract EEG data as a numpy array
labels = epochs.events[:, -1]    # Extract event IDs as labels


#======================================================== SVM =============================================

import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        
        clf = SVC(kernel='rbf', C=1.0)
        clf.fit(X_train_csp, y_train)
        predictions = clf.predict(X_val_csp)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)

# Binary Grey Wolf Optimization with 10-Fold Cross-Validation
def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            # Handling Hierarchy (Alpha, Beta, Delta)
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(alpha_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                # Encircling and Position Update Equations
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                #Binary Update with Sigmoid Function
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=4, max_iter=20
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")

None
None
Automatically created module for IPython interactive environment
Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-1_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 320479  =      0.000 ...   320.479 secs...


C:\Users\imran\AppData\Local\Temp\ipykernel_6356\1345623806.py:110: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_6356\1345623806.py:110: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-low_run-4_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 314999  =      0.000 ...   314.999 secs...


C:\Users\imran\AppData\Local\Temp\ipykernel_6356\1345623806.py:113: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
C:\Users\imran\AppData\Local\Temp\ipykernel_6356\1345623806.py:113: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Extracting parameters from C:\Users\imran\dataverse_files\eeg\sub-K\eeg\sub-K_task-mid_run-2_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 324079  =      0.000 ...   324.079 secs...


...

Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.
Not setting metadata


[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    1.8s


238 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 238 events and 600 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5', 'F3', 'F2', 'F4', 'F6']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8

C:\Users\imran\AppData\Local\Temp\ipykernel_6356\1345623806.py:295: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([target_epochs, non_target_epochs])


    Using tolerance 6e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.1e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 

In [43]:
#===================================================== Logistic Regression ==========================================
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        random_seed = 42
        clf = clf = LogisticRegression(max_iter=100)
        clf.fit(X_train_csp, y_train)
        predictions = clf.predict(X_val_csp)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)

# Binary Grey Wolf Optimization with 10-Fold Cross-Validation
def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(iteration_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=20, max_iter=2
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")


Computing rank from data with rank=None
    Using tolerance 6.5e-05 (2.2e-16 eps * 22 dim * 1.3e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.5e-05 (2.2e-16 eps * 22 dim * 1.3e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.5e-05 (2.2e-16 eps * 22 dim * 1.3e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covarian

In [3]:
#======================================================== SVM =============================================

import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        
        clf = SVC(kernel='rbf', C=1.0)
        clf.fit(X_train_csp, y_train)
        predictions = clf.predict(X_val_csp)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)

# Binary Grey Wolf Optimization with 10-Fold Cross-Validation
def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(iteration_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=20, max_iter=2
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")



Computing rank from data with rank=None
    Using tolerance 9.5e-05 (2.2e-16 eps * 22 dim * 1.9e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 9.4e-05 (2.2e-16 eps * 22 dim * 1.9e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 9.4e-05 (2.2e-16 eps * 22 dim * 1.9e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covarian

In [50]:
# ========================================= Polynomial Logistic Regression  =============================================
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
from mne.decoding import CSP

# Fitness function with 10-Fold Cross-Validation
def fitness_function(solution, X, y):
    selected_features = np.where(solution == 1)[0]
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features
    
    X_selected = X[:, selected_features]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        X_train_csp = csp.fit_transform(X_train, y_train)
        X_val_csp = csp.transform(X_val)
        
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train_csp)
        X_val_poly = poly.transform(X_val_csp)
        
        clf = LogisticRegression(max_iter=100)
        clf.fit(X_train_poly, y_train)
        predictions = clf.predict(X_val_poly)
        
        accuracies.append(accuracy_score(y_val, predictions))
    
    return np.mean(accuracies)

# Binary Grey Wolf Optimization with 10-Fold Cross-Validation
def binary_gwo(X, y, num_features, initial_wolves, num_wolves=20, max_iter=2):
    remaining_wolves = np.random.randint(0, 2, (num_wolves, num_features))
    wolves = np.vstack((initial_wolves, remaining_wolves))
    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []
    
    for iteration in range(max_iter):
        iteration_fitness = []
        for i, wolf in enumerate(wolves):
            fitness = fitness_function(wolf, X, y)
            iteration_fitness.append(fitness)
            
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness
        
        fitness_per_iteration.append(iteration_fitness)
        
        for i in range(num_wolves):
            for j in range(num_features):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3
                
                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])
                
                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta
                
                wolves[i, j] = (X1 + X2 + X3) / 3
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0
    
    return alpha, alpha_fitness, fitness_per_iteration

# Prepare Data (replace with actual EEG data)
X = epochs_data  # EEG dataset (n_samples, n_channels)
y = labels

num_features = X.shape[1]
initial_wolves = np.zeros((4, num_features))
lobe_channels = {
    "Frontal": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0, lobe_channels['Frontal']] = 1
initial_wolves[1, lobe_channels['Temporal']] = 1
initial_wolves[2, lobe_channels['Parietal']] = 1
initial_wolves[3, lobe_channels['Occipital']] = 1

# Run Binary GWO with 10-Fold CV
best_solution, best_fitness, fitness_iteration = binary_gwo(
    X, y, num_features, initial_wolves, num_wolves=20, max_iter=2
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration:", fitness_iteration)

best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)

selected_lobes = {}
for lobe, channels in lobe_channels.items():
    selected = np.intersect1d(best_selected_features, np.array(channels))
    selected_lobes[lobe] = selected.tolist()

print("\nSelected Lobes Information:")
for lobe, selected in selected_lobes.items():
    print(f"{lobe} lobe: Selected Channels: {selected}")


Computing rank from data with rank=None
    Using tolerance 9.5e-05 (2.2e-16 eps * 22 dim * 1.9e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 9.4e-05 (2.2e-16 eps * 22 dim * 1.9e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 9.4e-05 (2.2e-16 eps * 22 dim * 1.9e+10  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=-5100 covarian

C:\Users\imran\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    Using tolerance 0.0001 (2.2e-16 eps * 26 dim * 1.8e+10  max singular value)
    Estimated rank (data): 26
    data: rank 26 computed from 26 data channels with 0 projectors
Reducing data rank from 26 -> 26
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0001 (2.2e-16 eps * 26 dim * 1.8e+10  max singular value)
    Estimated rank (data): 26
    data: rank 26 computed from 26 data channels with 0 projectors
Reducing data rank from 26 -> 26
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0001 (2.2e-16 eps * 26 dim * 1.8e+10  max singular value)
    Estimated rank (data): 26
    data: rank 26 computed from 26 data channels with 0 projectors
Reducing data rank from 26 -> 26
Estimating class=-5100 covariance using EMPIRICAL
Done.
Estimating class=1

In [ ]:
#===================================================== Logistic Regression ==========================================

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import random
from mne.decoding import CSP

# Fitness function (Logistic Regression accuracy)
def fitness_function(solution, X_train, y_train, X_val, y_val):
    selected_features = np.where(solution == 1)[0]
    print("selected_features shape")
    print(selected_features.shape)
    print("selected_features")
    print(selected_features)
    if len(selected_features) == 0:
        return 0  # Avoid solutions with no selected features

    X_train_selected = X_train[:, selected_features]
    print("X_train_selected shape")
    print(X_train_selected.shape)



    X_val_selected = X_val[:, selected_features]
    print("X_val_selected shape")
    print(X_val_selected.shape)

    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
    print("csp.shape")


    csp_data_train = csp.fit_transform(X_train_selected, y_train)
    print("csp_data_train shape")
    print(csp_data_train.shape)
    print("csp_data_train shape")
    print(csp_data_train.shape)

    csp_data_val = csp.transform(X_val_selected)
    print("csp_data_val shape")
    print(csp_data_val.shape)
    print("csp_data_val shape")
    print(csp_data_val.shape)

    clf = LogisticRegression(max_iter=100)
    print("clf")
    print(clf)
    clf.fit(csp_data_train, y_train)
    print("clf.fit")
    print(clf.fit)
    predictions = clf.predict(csp_data_val)
    print("predictions shape")
    print(predictions.shape)
    print("predictions")
    print(predictions)

    return accuracy_score(y_val, predictions)

# Binary Grey Wolf Optimization (GWO) with Fitness Logging
def binary_gwo(X_train, y_train, X_val, y_val, num_features,initial_wolves, num_wolves=25, max_iter=3): # selected_channels=30
    # Initialize wolves' positions
    remaining_wolves = np.random.randint(0,2, (num_wolves, num_features))
    print("remaining_wolves shape")
    print(remaining_wolves.shape)
    print("remaining_wolves")
    print(remaining_wolves)

    wolves=np.vstack((initial_wolves,remaining_wolves))
    print("wolves shape")
    print(wolves.shape)
    print("wolves")
    print(wolves)

    alpha, beta, delta = None, None, None
    alpha_fitness, beta_fitness, delta_fitness = -1, -1, -1
    fitness_per_iteration = []

    # Main optimization loop
    for iteration in range(max_iter):
        iteration_fitness = []  # Store fitness of all wolves in the current iteration

        for i, wolf in enumerate(wolves):
            # Map wolf features to selected channels
            full_solution  = wolf
            print("full_solution shape")
            print(full_solution.shape)
            print("full_solution")
            print(full_solution)

            # Evaluate fitness
            fitness = fitness_function(full_solution, X_train, y_train, X_val, y_val)
            print("fitness")
            print(fitness)
            print()

            iteration_fitness.append(fitness)

            # Update alpha, beta, and delta wolves
            if fitness > alpha_fitness:
                delta, beta, alpha = beta, alpha, wolf.copy()
                delta_fitness, beta_fitness, alpha_fitness = beta_fitness, alpha_fitness, fitness
            elif fitness > beta_fitness:
                delta, beta = beta, wolf.copy()
                delta_fitness, beta_fitness = beta_fitness, fitness
            elif fitness > delta_fitness:
                delta = wolf.copy()
                delta_fitness = fitness

        # Log fitness for the current iteration
        fitness_per_iteration.append(iteration_fitness)
        print(f"Iteration {iteration + 1} Fitness Scores: {iteration_fitness}")

        # Update wolves' positions
        for i in range(num_wolves):
            for j in range(selected_channels):
                r1, r2, r3 = random.random(), random.random(), random.random()
                A1, A2, A3 = 2 * r1 - 1, 2 * r2 - 1, 2 * r3 - 1
                C1, C2, C3 = 2 * r1, 2 * r2, 2 * r3

                D_alpha = abs(C1 * alpha[j] - wolves[i, j])
                D_beta = abs(C2 * beta[j] - wolves[i, j])
                D_delta = abs(C3 * delta[j] - wolves[i, j])

                X1 = alpha[j] - A1 * D_alpha
                X2 = beta[j] - A2 * D_beta
                X3 = delta[j] - A3 * D_delta

                wolves[i, j] = (X1 + X2 + X3) / 3
                wolves[i, j] = 1 if random.random() < 1 / (1 + np.exp(-wolves[i, j])) else 0

    return alpha, alpha_fitness, fitness_per_iteration


# Prepare data (assuming epochs_data and labels are already loaded)
# Simulated placeholders: Replace with actual EEG data and labels
# epochs_data = ...
# labels = ...

# Split data into training and validation sets
X = epochs_data  # EEG dataset (n_samples, n_channels)
print("x.shape")
print(X.shape)

print()

y = labels
print("labels shape")
print(labels.shape)

print()

      # Corresponding labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print("X_train shape")
print(X_train.shape)

print()

print("X_val shape")
print(X_val.shape)

print()

print("y_train shape")
print(y_train.shape)

print()

print("y_val shape")
print(y_val.shape)

print()



# Define number of features and randomly selected channels
num_features = 64  # Number of EEG channels
selected_channels = 30
initial_wolves=np.zeros((4, num_features))
print("initial_wolves shape")
print(initial_wolves.shape)
print("initial_wolves")
print(initial_wolves)


# Group EEG channels by lobes
lobe_channels = {
    "Frontal": [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "Temporal": [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
    "Parietal": [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Occipital": [ 56, 57, 58, 59, 60, 61, 62, 63],
}

initial_wolves[0,lobe_channels['Frontal']]=1
initial_wolves[1,lobe_channels['Temporal']]=1
initial_wolves[2,lobe_channels['Parietal']]=1
initial_wolves[3,lobe_channels['Occipital']]=1

# Run Binary GWO with random 30 channel selection
best_solution, best_fitness, fitness_iteration = binary_gwo(X_train, y_train, X_val, y_val, num_features, initial_wolves,num_wolves=20, max_iter=3) #max_iter=1
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Fitness Iteration", fitness_iteration)

# Evaluate the best solution
best_selected_features = np.where(best_solution == 1)[0]
print("Best selected features shape:", best_selected_features.shape)
print("Best selected features (indices):", best_selected_features)
